# LangchainNotionToolkit

This notebook helps you get started with the **LangchainNotionToolkit**.  
It interacts with the Notion API to search, read, create, and update pages.  
For full options, see the API reference.

- API reference: https://pypi.org/project/langchain-notion/  
- Notion API docs: https://developers.notion.com/docs

## Overview

### Integration details

| Class | Package | Serializable | JS support | Package latest |
| :--- | :--- | :---: | :---: | :---: |
| `LangchainNotionToolkit` | [langchain-notion](https://pypi.org/project/langchain-notion/) | N/A | N/A | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-notion?style=flat-square&label=%20) |

### Tool features

The `LangchainNotionToolkit` bundles multiple tools for interacting with Notion through agents:

| Tool | Description | Input Schema |
| :--- | :--- | :--- |
| Search Pages | Search for pages in Notion using keywords. | `{"query": str}` |
| Get Page | Retrieve a page’s title, URL, and properties by ID. | `{"page_id": str}` |
| Create Page | Create a new page in a database or under another page. | `{"parent": {...}, "properties": {...}}` |
| Update Page | Update properties of an existing page. | `{"page_id": str, "properties": {...}}` |

## Setup
To use this toolkit you need a Notion integration token. Create one in Notion, then set:

```bash
export NOTION_API_KEY="your-notion-token"
```

Optionally enable LangSmith for tracing:

```bash
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

## Installation
Install the Notion integration package. Optionally install LangChain community extras.

```bash
%pip install -qU langchain-notion
%pip install -qU langchain-community
```

## Instantiation

By default you construct a `NotionWrapper` with your API key and then build a `LangchainNotionToolkit` from it.

```python
import os
from langchain_notion.notion_wrapper import NotionWrapper
from langchain_notion.toolkits import LangchainNotionToolkit

api = NotionWrapper(api_key=os.environ["NOTION_API_KEY"])
# include_write_tools=True exposes create/update tools in addition to read/search
toolkit = LangchainNotionToolkit.from_notion_wrapper(api, include_write_tools=True)
```
### View available tools
```python
tools = toolkit.get_tools()
tools
```

You should see entries like:
- Search Pages
- Get Page
- Create Page
- Update Page

## Invocation

### Invoke directly with args

Below is a simple example of calling the tool with keyword arguments in a dictionary.
```python
search_tool = next(t for t in tools if t.name == "Search Pages")
print(search_tool.invoke({"tool_input":"test"}))
```
### Invoke with ToolCall

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:
```python
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"tool_input":"test"}, 
    "id": "1",
    "name": search_tool.name,
    "type": "tool_call",
}
search_tool.invoke(model_generated_tool_call)
```

## Customizing Authentication

Behind the scenes the toolkit uses the official Notion client.  
If you need custom construction, pass your token directly or from a secrets store.
```python
from langchain_notion.notion_wrapper import NotionWrapper

# Example: explicit token string (prefer env vars or a secret manager in practice)
custom_api = NotionWrapper(api_key=os.environ["NOTION_API_KEY"])
custom_toolkit = LangchainNotionToolkit.from_notion_wrapper(custom_api, include_write_tools=True)
```

## Use within an agent

Below shows how to incorporate the toolkit into an agent and let the model decide when to call tools.
```python
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent_executor = create_react_agent(llm, tools)

example_query = (
    "Search Notion for pages about 'roadmap'. "
    "If you find one, get details for the top result."
)

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

```
_Example output (will vary):_
```
================================ Human Message =================================
Search Notion for pages about 'roadmap'. If you find one, get details for the top result.
================================== Ai Message ==================================
Tool Calls:
  Search Pages (...)
  Get Page (...)
================================= Tool Message =================================
Name: Search Pages
Found 3 results. Top: "2025 Product Roadmap" (page_id=...)
================================= Tool Message =================================
Name: Get Page
Title: 2025 Product Roadmap
URL: https://www.notion.so/...
Properties: {...}
================================== Ai Message ==================================
Top page is "2025 Product Roadmap". I’ve included its URL and key properties.
```

## Direct invocation examples

### Search pages
```python
search_tool = next(t for t in tools if t.name == "Search Pages")
search_tool.invoke({"tool_input": "roadmap"})

### Get a page by ID
get_tool = next(t for t in tools if t.name == "Get Page")
# Replace with an actual page ID you have access to
# get_tool.invoke({"tool_input": {"page_id": "XXXXXXXXXXXX"}})

### Create a page (requires include_write_tools=True)
create_tool = next(t for t in tools if t.name == "Create Page")
example_create_args = {
    "tool_input": {
        "parent": {"database_id": "YOUR_DATABASE_ID"},
        "properties": {
            "Name": {"title": [{"text": {"content": "Project Alpha"}}]},
            # Add other database properties as needed
        },
    }
}
# create_tool.invoke(example_create_args)


### Update a page

update_tool = next(t for t in tools if t.name == "Update Page")
example_update_args = {
    "tool_input": {
        "page_id": "YOUR_PAGE_ID",
        "properties": {
            "Status": {"select": {"name": "In Progress"}},
        },
    }
}
# update_tool.invoke(example_update_args)
```

## API reference

For detailed documentation of all **LangchainNotionToolkit** features and configurations:

- https://github.com/rujeetjahagirdar/langchain-notion
- https://pypi.org/project/langchain-notion/